In [28]:
import os

os.chdir('/home/yry08/yigedabuliu/lkq/LiteMedSAM')

In [29]:
brats_data_root = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/h5_datasets/'

### 需要将数据集转换为形状为 img:[3,H,W]   gt:[1,H,W]  box:[1,1,4]

In [30]:
# 导入相关包
import torch
from tqdm import tqdm
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [31]:
# 单张截面转换测试

def single_convert(single_img, single_label):
    """
        将单张[H,W]的图像和标签改为合适的尺寸
        single_img: tensor [240,240]
        single_label: tensor [240,240]
    """

    # 将通道数堆叠为3
    single_img = np.stack((single_img,) * 3, axis=-1)
    single_img = np.clip(single_img, 0, 1)

    # 只保留原始标签为1的，即NT区域
    NT_label = single_label.copy()
    NT_mask = NT_label != 1
    NT_label[NT_mask] = 0

    # 只保留原始标签为2的，即ED区域
    ED_label = single_label.copy()
    ED_mask = ED_label != 2
    ED_label[ED_mask] = 0

    # 只保留原始标签为4的，即ET区域
    ET_label = single_label.copy()
    ET_mask = ET_label != 4
    ET_label[ET_mask] = 0

    # single_img = np.transpose(single_img, (2, 0, 1))
    NT_label = np.expand_dims(NT_label, axis=0)
    ED_label = np.expand_dims(ED_label, axis=0)
    ET_label = np.expand_dims(ET_label, axis=0)

    return single_img, NT_label, ED_label, ET_label

In [ ]:
# 读取并转换数据

save_root = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/npy/'

all_sequence = sorted(os.listdir(brats_data_root))

modal_size = 4

for sequence in tqdm(all_sequence):
    # 读取每个h5文件
    h5_path = brats_data_root + sequence

    h5f = h5py.File(h5_path, 'r')
    image = h5f['image'][:]
    label = h5f['label'][:]

    modal = image.shape[0] # 4个模态
    depth = image.shape[-1] # 155个帧

    for i in range(modal):
        cur_modal = '_modal_' + str(i)
        # print(f'cur modal:{cur_modal}')
        for j in range(depth):
            img_f = image[i, :, :, j].copy()
            label_f = label[:, :, j].copy()

            img_f, NT_label, ED_label, ET_label = single_convert(img_f, label_f)

            # BraTS2021_00000_mri_norm2_modal_2_ET_49


            # 保存非空NT
            if np.sum(NT_label) != 0:

                frame_name = sequence.split('.')[0] + cur_modal + '_NT_' + str(j) + '.npy'
                # print(f'frame name:{frame_name}')
                img_save_path = save_root + 'imgs/' + frame_name
                label_save_path = save_root + 'gts/' + frame_name
                np.save(img_save_path, img_f)
                np.save(label_save_path, NT_label)

            # 保存非空ED
            if np.sum(ED_label) != 0:
                frame_name = sequence.split('.')[0] + cur_modal + '_ED_' + str(j) + '.npy'
                # print(f'frame name:{frame_name}')
                img_save_path = save_root + 'imgs/' + frame_name
                label_save_path = save_root + 'gts/' + frame_name
                np.save(img_save_path, img_f)
                np.save(label_save_path, ED_label)

            # 保存非空ET
            if np.sum(ET_label) != 0:
                frame_name = sequence.split('.')[0] + cur_modal + '_ET_' + str(j) + '.npy'
                # print(f'frame name:{frame_name}')
                img_save_path = save_root + 'imgs/' + frame_name
                label_save_path = save_root + 'gts/' + frame_name
                np.save(img_save_path, img_f)
                np.save(label_save_path, ET_label)



 83%|███████████████████████████████████████████████████████████████▌             | 1033/1251 [1:24:44<19:28,  5.36s/it]

In [35]:
image.shape

(4, 240, 240, 155)